In [1]:
import torch
import torch.nn.functional as F
import torchvision
from torch import nn

定义基础卷积模型 ： 卷积+BN+激活

In [2]:
class BasicConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,**kwargs):
        super(BasicConv2d,self).__init__()
        self.conv = nn.Conv2d(in_channels,out_channels,bias=False,**kwargs)
        self.bn   = nn.BatchNorm2d(out_channels)
    
    def forward(self,x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x,inplace=True)
        return x

![jupyter](Inception.png)

In [3]:
class InceptionBlock(nn.Module): # InceptionBlock架构如上图所示
    def __init__(self,in_channels,pool_features):
        super(InceptionBlock,self).__init__()
        
        self.b_1x1   = BasicConv2d(in_channels,64,kernel_size=(1,1)) # 设置一个具有1X1大小卷积核的卷积层
        
        self.b_3X3_1 = BasicConv2d(in_channels,64,kernel_size=(1,1))
        self.b_3X3_2 = BasicConv2d(64,96,kernel_size=(3,3),padding=1)# 每一次卷积，图像周围会减少k-1个像素
        
        self.b_5X5_1 = BasicConv2d(in_channels,48,kernel_size=(1,1))
        self.b_5X5_2 = BasicConv2d(48,64,kernel_size=(5,5),padding=2)# 每一次卷积，图像周围会减少k-1个像素
        
        self.b_pool  = BasicConv2d(in_channels,pool_features,kernel_size=1)
    
    def forward(self,input):
        b_1x1_out = self.b_1x1(input)
        
        b_3x3     = self.b_3X3_1(input)
        b_3x3_out = self.b_3X3_2(b_3X3)
        
        b_5x5     = self.b_5X5_1(input)
        b_5x5_out = self.b_5X5_2(b_5x5)
        
        b_pool_out= F.max_pool2d(x,kernel_size=(3,3),stride=1,padding=1)
        b_pool_out= self.b_pool(b_pool_out)
        
        outputs = [b_1x1_out,b_3x3_out,b_5x5_out,b_pool_out]
        
        return torch.cat(outputs,dim=1) #将各个层的输出按照纵向按值相加

In [4]:
my_inception_block = InceptionBlock(32,64)